In [ ]:
import random
import time
import sympy
import pandas as pd
from math import isqrt

##Pembangkitan Basis Parameter

In [ ]:
def generate_Rx_y():
    # Inisialisasi matriks Rx_y dengan nilai awal
    Rx_y = [[(0, 0) for _ in range(9)] for _ in range(9)]

    # Mengisi matriks Rx_y dengan nilai yang diurutkan
    current_ash = -4
    current_bsh = -4
    for i in range(9):
        for j in range(9):
            Rx_y[i][j] = (current_ash, current_bsh)
            current_bsh += 1
            if current_bsh > 4:
                current_bsh = -4
                current_ash += 1

    # Memastikan nilai-nilai berbeda dalam setiap baris dan kolom
    for i in range(9):
        for j in range(9):
            while Rx_y[i][j] in Rx_y[i][:j] + Rx_y[i][j+1:]:
                # Jika nilai sudah ada, geser ke nilai selanjutnya
                current_bsh += 1
                if current_bsh > 4:
                    current_bsh = -4
                    current_ash += 1
                Rx_y[i][j] = (current_ash, current_bsh)

            # Pastikan nilai berbeda dalam kolom yang sama
            for k in range(i):
                while Rx_y[i][j] in (Rx_y[l][j] for l in range(k)):
                    # Jika nilai sudah ada, geser ke nilai selanjutnya
                    current_bsh += 1
                    if current_bsh > 4:
                        current_bsh = -4
                        current_ash += 1
                    Rx_y[i][j] = (current_ash, current_bsh)

    return Rx_y

Rx_y = generate_Rx_y()
print(Rx_y)

[[(-4, -4), (-4, -3), (-4, -2), (-4, -1), (-4, 0), (-4, 1), (-4, 2), (-4, 3), (-4, 4)], [(-3, -4), (-3, -3), (-3, -2), (-3, -1), (-3, 0), (-3, 1), (-3, 2), (-3, 3), (-3, 4)], [(-2, -4), (-2, -3), (-2, -2), (-2, -1), (-2, 0), (-2, 1), (-2, 2), (-2, 3), (-2, 4)], [(-1, -4), (-1, -3), (-1, -2), (-1, -1), (-1, 0), (-1, 1), (-1, 2), (-1, 3), (-1, 4)], [(0, -4), (0, -3), (0, -2), (0, -1), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4)], [(1, -4), (1, -3), (1, -2), (1, -1), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4)], [(2, -4), (2, -3), (2, -2), (2, -1), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4)], [(3, -4), (3, -3), (3, -2), (3, -1), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4)], [(4, -4), (4, -3), (4, -2), (4, -1), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4)]]


In [ ]:
def random_big_integer(bits):
    return random.getrandbits(bits)

def random_range(start, end):
    return random.randint(start, end)

def dec_to_bin(y):
    return format(y, 'b')

def generate_S(l, Rx_y):
    start_time = time.time()
    S = [[0 for _ in range(2)] for _ in range(81)]
    K = random_big_integer(l - 63)
    a = random_big_integer(l)
    b = random_big_integer(l)
    while abs(len(dec_to_bin(a))-len(dec_to_bin(b))) > 4:
      a = random_big_integer(l)
      b = random_big_integer(l)
    S[0] = [a,b]

    for i in range(1, 81):
        Dsh = random_range(32, 54)
        Z = K << Dsh
        x = i // 9
        y = i % 9
        ash, bsh = Rx_y[x][y]

        if ash >= 0:
            S[i][0] = (a << ash) + Z
        else:
            S[i][0] = (a >> -ash) + Z

        if bsh >= 0:
            S[i][1] = (b << bsh) + Z
        else:
            S[i][1] = (b >> -bsh) + Z

    Du = random_range(2**16, 2**32)
    end_time = time.time()
    return S, Du, end_time - start_time

# Contoh Penggunaan
S, Du, exec_time = generate_S(126, Rx_y)
print("S matrix:", S)
print("Du:", Du)
print("Execution Time:", exec_time)

S matrix: [[42384170098262508537696793706111925260, 22765000742899512456171274536675715973], [2649011570197185413436771368814032832, 2845626031918217686852131079266502000], [2649011570197185413436771368814032832, 5691251124780656743873540396350966497], [2649025656033864860897597801544595392, 11382515396342214305377185463250458050], [2649070730711239092772242386282395584, 22765060842469344765337467316326116229], [2649040680926322938189145996457195456, 45530031535583941066925645463176632074], [2649070730711239092772242386282395584, 91060063071167882133851290926353264148], [2649010689832392947970469716768372672, 182120006001887085813734616403542105128], [2649010645814153324697154634166089664, 364240011901064945839831497614345549904], [5298051312067729721795195603089190785, 1422842596216135683093801048367432376], [5298261660562142803876870331865591681, 2845865491141768293686180435686065520], [5301867634752081353848437110889607041, 5695096558194145900679156531794545377], [529802877472904260

##Pembangkitan Domain Kunci RSA

In [ ]:
def fpb(a, b):
    """Menghitung Faktor Persekutuan Terbesar (FPB) dari dua bilangan."""
    while b:
        a, b = b, a % b
    return abs(a)

def modinv(a, m):
    """Menghitung invers modulo a^(-1) mod m dengan menggunakan algoritma Extended Euclidean."""
    if fpb(a,m)==1:
      m0, x0, x1 = m, 0, 1
      while a > 1:
          q = a // m
          m, a = a % m, m
          x0, x1 = x1 - q * x0, x0
      return x1 + m0 if x1 < 0 else x1
    else:
      print(f"{a} tidak relatif prima dengan {m}!")

In [ ]:
def Ga(id, time):
    # Fungsi placeholder Ga untuk menghasilkan nilai da
    da = hash((id, time, "Ga")) % 103
    if da == 0:
      return 103
    else:
      return da

def Gb(id, time):
    # Fungsi placeholder Gb untuk menghasilkan nilai db
    db = hash((id, time, "Gb")) % 103
    if db == 0:
      return 103
    else:
      return db

def is_probable_prime(n, k=8):
    # Algoritma Miller-Rabin dengan banyak iterasi k=8 untuk mengecek keprimaan suatu bilangan
    if n == 2 or n == 3:
        return True
    if n % 2 == 0 or n == 1:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2

    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True

def NextPrime(x):
    x = x + 1 if x % 2 == 0 else x + 2
    while True:
        if is_probable_prime(x):
            return x
        x += 2

def generate_keys(k, S, Du, e, da, db):
    start_time = time.time()
    l = len(dec_to_bin(S[0][1]))   # Bitsize of S[0][1]
    u = random_big_integer((k // 2) - l)
    m = random_range(0, 80)

    p, q = 0, 0

    for i in range(81):
        f = (m + i) % 81
        d = (u - Du) * (u + Du) * S[f][0] * S[f][1]

        if len(dec_to_bin(d)) == k:
            rr_a = random_range(1, 171)
            Da = rr_a << da
            p = NextPrime((u - Du) * (S[f][0] + Da))
            rr_b = random_range(1, 171)
            Db = rr_b << db
            q = NextPrime((u + Du) * (S[f][1] + Db))

            if p < q:
              p, q = q, p

            if fpb(e, (p - 1) * (q - 1)) == 1:
              break

    N = p * q
    d = modinv(e, (p - 1) * (q - 1))
    end_time = time.time()

    return (N,e), (p, q, d), end_time - start_time

# Contoh Penggunaan
e = 65537
id_cert = "sample_id"
start_date = "2024-05-10"
da = Ga(id_cert, start_date)
db = Gb(id_cert, start_date)

public_key, private_key, execution_time = generate_keys(1024, S, Du, e, da, db)
print("Ukuran Kunci:", 1024, "bit")
print("Kunci Publik (n, e):", public_key)
print("Kunci Privat (p, q, d):", private_key)
print("Waktu Eksekusi:", execution_time, "detik")

Ukuran Kunci: 1024 bit
Kunci Publik (n, e): (93785352808783606863670653030360357711019623003314873154506414843053242907246751515657453376052439180130278497499476982800361053588033211520772416643772033217231607075984008786142468032812697466386072787550671346053721654926599926870056318616609017807832452394846990687622905654594311933017121616061213103889, 65537)
Kunci Privat (p, q, d): (211353332151875321401947310559861703530542186421089503122602213700184391825242102914029180158804155488307771610179559893712812160433568481267198963743083621, 443737280382174740389717892930204048348911792476418036066666308442345237061334932719355728016821311767446069697761716891760441248986253945407047586965309, 278893257379859272497379702596806843680678308850207297085337369558610350034229815262927523207636295097663762094353923267436140893446019692590190827772169770916816550198744225281410965260844996858840820966833631775247340159968551923809501855639286951499151624091095087109604305321719693295899434624237

## Faktorisasi Kunci RSA

In [ ]:
def factor_keys(S, N, e, da, db):
    start_time = time.time()
    for index in range(81):
        for wL in range(1, 172):
            aV = S[index][0] + (wL << da)

            for wH in range(1, 172):
                bV = S[index][1] + (wH << db)
                n = aV * bV
                M = n * N
                t = isqrt(M)
                z = (t+1)**2 - M
                i = isqrt(z)

                if i**2 == z:
                    f = t - i + 1
                    q = fpb(f, N)
                    p = N // q
                    d = modinv(e, (p - 1) * (q - 1))
                    end_time = time.time()
                    return p, q, d, end_time-start_time
    end_time = time.time()
    return 0, 0, 0, end_time-start_time

p, q, d, execution_time = factor_keys(S, public_key[0], public_key[1], da, db)
print("Prima p dan q untuk ukuran ", 1024, " bit :", (p,q))
print("Waktu Eksekusi:", execution_time, "detik")

Prima p dan q untuk ukuran  1024  bit : (443737280382174740389717892930204048348911792476418036066666308442345237061334932719355728016821311767446069697761716891760441248986253945407047586965309, 211353332151875321401947310559861703530542186421089503122602213700184391825242102914029180158804155488307771610179559893712812160433568481267198963743083621)
Waktu Eksekusi: 16.035181522369385 detik
